In [3]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 35.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1b686f08178ffac52c4fb03214e82bd00d273d5f961ff0c81720842838b3ae55
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Del ejercicio de M&M aplicar: 


1.   Otras operaciones de agregación como el Max con otro tipo de 
ordenamiento (descendiente).
2.   Hacer un ejercicio como el “where” de CA que aparece en el libro pero 
indicando más opciones de estados (p.e. NV, TX, CA, CO).
3. Hacer un ejercicio donde se calculen en una misma operación el Max, 
Min, Avg, Count. Revisar el API (documentación) donde encontrarán 
este ejemplo:
ds.agg(max($"age"), avg($"salary"))
ds.groupBy().agg(max($"age"), avg($"salary"))
NOTA: $ es un alias de col()
4. Hacer también ejercicios en SQL creando tmpView

In [13]:
from pyspark.sql import SparkSession

#Create SparkSession
spark = (SparkSession
  .builder
  .appName("PythonMnMCount")
  .getOrCreate())
#Read dataset
mnm_df = (spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load('/content/drive/My Drive/Bosonit pyspark/data/mnm_dataset.csv'))

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, desc, asc, avg

def mnm_op():
  
#Select columns State, Color and Count, grouped by State and Color, aggregate Count with average, alias "Avg", order by "Avg"
  op_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .groupBy("State", "Color")
    .agg(avg("Count").alias("Avg"))
    .orderBy("Avg", ascending=True))
#Show 60 rows without any truncate and print the total of rows
  op_mnm_df.show(n=60, truncate=False)
  print("Total Rows = %d" % (op_mnm_df.count()))
#Same as before but only from California
  ca_op_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .where(mnm_df.State == "CA")
    .groupBy("State", "Color")
    .agg(avg("Count").alias("Avg"))
    .orderBy(desc("Avg")))
#Show only 10 rows without truncate
  ca_op_mnm_df.show(n=10, truncate=False)


In [15]:
mnm_op()

+-----+------+------------------+
|State|Color |Avg               |
+-----+------+------------------+
|NV   |Green |53.78739693757362 |
|NV   |Blue  |53.797369994022716|
|WY   |Yellow|53.997539975399754|
|UT   |Red   |54.163690476190474|
|NM   |Green |54.1973840665874  |
|OR   |Orange|54.22489959839358 |
|WA   |Green |54.23608768971332 |
|UT   |Orange|54.24049881235155 |
|UT   |Yellow|54.263829787234044|
|CA   |Green |54.268717353453276|
|AZ   |Orange|54.28300769686205 |
|AZ   |Brown |54.350412249705535|
|UT   |Blue  |54.366767371601206|
|CA   |Orange|54.502715751357876|
|UT   |Brown |54.55119558553035 |
|NV   |Yellow|54.561194029850746|
|OR   |Yellow|54.60285006195787 |
|AZ   |Red   |54.637135922330096|
|WY   |Blue  |54.68870192307692 |
|CO   |Green |54.71336835960304 |
|NM   |Orange|54.8054054054054  |
|TX   |Blue  |54.811648079306075|
|OR   |Green |54.821297429620564|
|AZ   |Green |54.82219570405728 |
|NV   |Orange|54.865070093457945|
|OR   |Red   |54.88510638297873 |
|NM   |Yellow|

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, desc, asc, avg,count
def mnm_multiple_states():
#Select columns State, Color and Count where state in [CA, WA, NV, TX], grouped by State and Color, aggregate Count with count, alias "Total", order by "Total"
  multi_sats_op_mnm_df = (mnm_df
    .where((mnm_df.State == "CA") | (mnm_df.State == "WA") | (mnm_df.State == "NV") | (mnm_df.State == "TX"))
    .groupBy("Color", "State")
    .agg(count("Count").alias("Total"))
    .orderBy(desc("Total"))
    .select("State", "Color", "Total"))
#Show 60 rows without truncate
  multi_sats_op_mnm_df.show(n=60, truncate=False)


In [19]:
mnm_multiple_states()

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CA   |Brown |1718 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WA   |Brown |1669 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|NV   |Brown |1657 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|TX   |Orange|1652 |
|TX   |Brown |1641 |
|WA   |Blue  |1625 |
|TX   |Blue  |1614 |
|NV   |Red   |1610 |
|CA   |Blue  |1603 |
+-----+------+-----+



In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, desc, asc, avg,count
def mnm_multiple_agg():
  #Select columns State, Color, Total, Max, Avg, grouped by State and Color, aggregate Count with (count alias "Total", max alias "Max", avg alias "Avg"), order by "State"
  multi_agg_op_mnm_df = (mnm_df
    .groupBy("State", "Color")
    .agg(count("Count").alias("Total"), max("Count").alias("Max"),avg("Count").alias("Avg"))
    .orderBy(desc("State"))
    .select("State", "Color", "Total", "Max", "Avg"))
#Show 60 without truncate
  multi_agg_op_mnm_df.show(n=60, truncate=False)


In [23]:
mnm_multiple_agg()

+-----+------+-----+---+------------------+
|State|Color |Total|Max|Avg               |
+-----+------+-----+---+------------------+
|WY   |Brown |1532 |100|56.20757180156658 |
|WY   |Orange|1595 |100|55.144827586206894|
|WY   |Blue  |1664 |100|54.68870192307692 |
|WY   |Yellow|1626 |100|53.997539975399754|
|WY   |Green |1695 |100|55.657227138643066|
|WY   |Red   |1670 |100|54.950898203592814|
|WA   |Red   |1671 |100|55.85397965290245 |
|WA   |Green |1779 |100|54.23608768971332 |
|WA   |Blue  |1625 |100|55.314461538461536|
|WA   |Brown |1669 |100|55.77112043139604 |
|WA   |Yellow|1663 |100|55.8749248346362  |
|WA   |Orange|1658 |100|55.199638118214715|
|UT   |Orange|1684 |100|54.24049881235155 |
|UT   |Brown |1631 |100|54.55119558553035 |
|UT   |Red   |1680 |100|54.163690476190474|
|UT   |Blue  |1655 |100|54.366767371601206|
|UT   |Yellow|1645 |100|54.263829787234044|
|UT   |Green |1591 |100|55.557510999371466|
|TX   |Green |1737 |100|55.12550374208405 |
|TX   |Orange|1652 |100|55.88075

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, desc, asc, avg,count
def mnm_pysql():
  mnm_df.createOrReplaceTempView("mnm_view") 
  #Select columns State, Color and Count where State in [CA, TX, WA], grouped by State and Color, aggregate Count with average, order by "State"
  sql_mnm_view = spark.sql("select State, Color, avg(Count) from mnm_view \
        where State = 'CA' or State = 'TX' or State = 'WA' group by State, Color order by State;") 
  #Show 60 without truncate
  sql_mnm_view.show(n=60, truncate=False)


In [29]:
mnm_pysql()

+-----+------+------------------+
|State|Color |avg(Count)        |
+-----+------+------------------+
|CA   |Brown |55.740395809080326|
|CA   |Green |54.268717353453276|
|CA   |Blue  |55.59762944479102 |
|CA   |Yellow|55.8693967902601  |
|CA   |Orange|54.502715751357876|
|CA   |Red   |55.26992753623188 |
|TX   |Red   |55.306666666666665|
|TX   |Yellow|55.09042865531415 |
|TX   |Orange|55.880750605326874|
|TX   |Brown |55.29311395490554 |
|TX   |Blue  |54.811648079306075|
|TX   |Green |55.12550374208405 |
|WA   |Orange|55.199638118214715|
|WA   |Blue  |55.314461538461536|
|WA   |Red   |55.85397965290245 |
|WA   |Brown |55.77112043139604 |
|WA   |Green |54.23608768971332 |
|WA   |Yellow|55.8749248346362  |
+-----+------+------------------+



In [30]:
# Stop the SparkSession
spark.stop()